
# SolarX example.

This notebook demostrate an example for evaluating a solarX project using `hydesign`'s `hpp_model_solarX` class.  

It loads site data and simulation parameters, runs an evaluation, prints the design, and plots results.

## Introduction to SOLARX

**SOLARX** is an integrated renewable energy concept that combines three advanced high-concentration solar technologies to maximize efficiency and flexibility. The system can simultaneously deliver electricity, heat, and green hydrogen, either directly or via storage, enabling carbon-neutral and even carbon-negative operation.

Key innovations include:

* Smart solar resource management aligned with demand and market prices.
* High-efficiency concentrated photovoltaic (CPV) receivers.
* Advanced concentrated solar thermal (CST) receivers for power and heat.
* A carbon-negative bi-energy hydrogen receiver.

## Imports

Install hydesign if needed.

In [ ]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

Import basic libraries.

In [ ]:
import os
import time
from hydesign.assembly.hpp_assembly_solarX import hpp_model_solarX as hpp_model
from hydesign.examples import examples_filepath

In [ ]:
# Enable inline plots in notebooks
%matplotlib inline

## Initialize model

Site info

In [ ]:
longitude = -6.03
latitude = 37.380
altitude = 10


Input files

In [ ]:
#time series
input_ts_fn = os.path.join(examples_filepath, 'Europe/solarX_Spain.csv')

# YAML simulation parameters
sim_pars_fn = os.path.join(examples_filepath, 'Europe/hpp_pars_solarX.yml')

Initialized an hpp model

In [ ]:
batch_size = 1 * 24  # 3 months of hourly data per batch (example)
hpp = hpp_model(
    latitude=latitude, 
    longitude=longitude, 
    altitude=altitude,      # Geographical data
    work_dir='./',                      # Output directory
    sim_pars_fn=sim_pars_fn,            # Simulation parameters
    input_ts_fn=input_ts_fn,            # Input time series
    batch_size=batch_size,
    tower_num=25,                     # Number of towers in the plant
)
print("Model initialized.")



## Run evaluation

In this section, a single evaluation with the chosen sizing variables is performed.

Set Sizing variables

In [ ]:
# Solar field (sf)
sf_area = 8000
tower_height = 50

# Concentrated Photovoltaics (cpv)
area_cpv_receiver_m2 = 4

# Concentrated Solar Thermal (cst)
heat_exchanger_capacity = 50
p_rated_st = 15
v_molten_salt_tank_m3 = 2300
area_cst_receiver_m2 = 9

# Biogas + H2 (set to 0 as in your script)
area_dni_reactor_biogas_h2 = 0
area_el_reactor_biogas_h2 = 0

x = [
    # sf
    sf_area,
    tower_height,
    # cpv
    area_cpv_receiver_m2,
    # cst
    heat_exchanger_capacity,
    p_rated_st,
    v_molten_salt_tank_m3,
    area_cst_receiver_m2,
    # biogas_h2
    area_dni_reactor_biogas_h2,
    area_el_reactor_biogas_h2,
]


Run EMS evaluation

In [ ]:
print("Design vector x:", x)

start = time.time()
outs = hpp.evaluate(*x)  # Run the model evaluation
hpp.print_design()
end = time.time()

print(f'Execution time [min]:', round((end - start) / 60, 2))

### Plotting

In [ ]:
fig = hpp.plot_solarX_results(n_hours=7*24, index_hour_start=4344)  # `index_hour_start=4344` corresponds to **July 1** in an hourly series (0-indexed).
